# Import Module

In [1]:
# import module
import os
import glob
import time
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from IPython.display import clear_output

import torch
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
# for loss function
from typing import Optional
#from kornia.utils.one_hot import one_hot

from models import DP_UNet, DP_resUNet
from patch_function import makepatches_overlay, \
makepatches_overlay_normap, \
patchesback_overlay, \
find_ksp_andpadding, \
pad_back, \
pad_for_256, \
pad_back_256
from data_preprocess_and_loader import Dataset, Dataset_val
from losses_unet3d import DiceLoss, GeneralizedDiceLoss, compute_per_channel_dice
from train_and_val import train, validation

print(torch.cuda.is_available())
print(torch.__version__)

ModuleNotFoundError: No module named 'torch'

# Define Datasets

In [ ]:
#TWCC
train_image_path = '/work/nymu834wklee/TVGH/brain_tumor_nomask_biparametric_twcc4/imagesTr'
val_image_path = '/work/nymu834wklee/TVGH/brain_tumor_nomask_biparametric_twcc4/imagesVal'

k = os.listdir(train_image_path)
list_tra = []
for j in k:
    list_tra.append(train_image_path + '/' + j)
list_val = []
k = os.listdir(val_image_path)
for j in k:
    list_val.append(val_image_path + '/' + j)

train_image_path = '/work/nymu834wklee/TCVGH/brain_tumor_nomask_biparametric_twcc4/imagesTr'
val_image_path = '/work/nymu834wklee/TCVGH/brain_tumor_nomask_biparametric_twcc4/imagesVal'

k = os.listdir(train_image_path)
for j in k:
    list_tra.append(train_image_path + '/' + j)
k = os.listdir(val_image_path)
for j in k:
    list_val.append(val_image_path + '/' + j)

train_image_path = '/work/nymu834wklee/BraTS/raw_without_nonenhance_twcc4/imagesTr' #path that save the training data
val_image_path = '/work/nymu834wklee/BraTS/raw_without_nonenhance_twcc4/imagesVal'

k = os.listdir(train_image_path)
for j in k:
    list_tra.append(train_image_path + '/' + j)
k = os.listdir(val_image_path)
for j in k:
    list_val.append(val_image_path + '/' + j)

train_image_path = '/work/nymu834wklee/UCSF/raw_without_nonenhance_twcc4/imagesTr' #path that save the training data
val_image_path = '/work/nymu834wklee/UCSF/raw_without_nonenhance_twcc4/imagesVal'

k = os.listdir(train_image_path)
for j in k:
    list_tra.append(train_image_path + '/' + j)
k = os.listdir(val_image_path)
for j in k:
    list_val.append(val_image_path + '/' + j)      

In [3]:
# 設置dataloader
#biparametric = True
train_data = Dataset(list_tra, biparametric=True)
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=2,
    num_workers=8,
    shuffle = True)

val_data = Dataset_val(list_val, biparametric=True)
val_loader = torch.utils.data.DataLoader(
    val_data,           
    batch_size=2,          
    num_workers=8,
    shuffle = False)

# Initialize the Model

In [4]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialization

model = DP_resUNet(img_channels = 2, n_classes = 1).cuda()
model= nn.DataParallel(model)
model.to(device)

#model = DP_UNet(img_channels = 2, n_classes = 2).cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
criterion_DICE = DiceLoss()
criterion_GDL = GeneralizedDiceLoss()

loss_fun = (criterion_DICE,criterion_GDL,compute_per_channel_dice)

# Train the Model

In [5]:
# Train the Model
epochs = 2 # The number of epochs

# 儲存權重的路徑
path = 'F:/20220309_process4FL/baseline_model/aug/raw/pytorch_test'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

#f = open(path + '/loss_curve.txt', 'a')
# 開始訓練
traloss = 0
valloss = 0
rvalloss = 0
for epoch in range(1,epochs+1):
    if epoch==1:
        print('Training start.')
    
    epoch_start_time = time.time()
    tralossnew = train(train_loader,model,loss_fun,optimizer,epoch,path)
    print('-' * 89)
    vallossnew = validation(val_loader,model,loss_fun,epoch,path)
    print('-' * 89)    
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    #scheduler.step()
    # save best model parameters
    f2 = open(path + '/model_info.txt', 'a')
    if tralossnew<traloss or epoch ==1: 
        fname = path + '/best_tra'  + '.tar'
        torch.save(model.state_dict(), fname)
        traloss = tralossnew
        f2.write('| best_tra | epoch {:3d}| '.format(epoch)+'\r\n')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')
        

fname = path + '/last' + '.tar'
torch.save(model.state_dict(), fname)
f2.write('Finished!')
    
#f.close()
f2.close()
clear_output(wait=True)

Training start.
| epoch   1 |   100/ 3779 batches |  ms/batch 53025.93 |  loss_dice  0.98  |  loss_gdl  0.98  |  dice_score  0.02  | 
| epoch   1 |   200/ 3779 batches |  ms/batch 43018.03 |  loss_dice  0.97  |  loss_gdl  0.97  |  dice_score  0.03  | 
| epoch   1 |   300/ 3779 batches |  ms/batch 41040.20 |  loss_dice  0.96  |  loss_gdl  0.96  |  dice_score  0.04  | 
| epoch   1 |   400/ 3779 batches |  ms/batch 47394.27 |  loss_dice  0.96  |  loss_gdl  0.96  |  dice_score  0.04  | 
| epoch   1 |   500/ 3779 batches |  ms/batch 45027.48 |  loss_dice  0.95  |  loss_gdl  0.95  |  dice_score  0.05  | 
| epoch   1 |   600/ 3779 batches |  ms/batch 39054.74 |  loss_dice  0.95  |  loss_gdl  0.95  |  dice_score  0.05  | 
| epoch   1 |   700/ 3779 batches |  ms/batch 46712.45 |  loss_dice  0.94  |  loss_gdl  0.94  |  dice_score  0.06  | 
| epoch   1 |   800/ 3779 batches |  ms/batch 43758.56 |  loss_dice  0.94  |  loss_gdl  0.94  |  dice_score  0.06  | 
| epoch   1 |   900/ 3779 batches |  ms/

| epoch   2 |  2500/ 3779 batches |  ms/batch 46747.47 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  2600/ 3779 batches |  ms/batch 42133.20 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  2700/ 3779 batches |  ms/batch 55628.03 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  2800/ 3779 batches |  ms/batch 41728.61 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  2900/ 3779 batches |  ms/batch 41858.95 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  3000/ 3779 batches |  ms/batch 46878.71 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  3100/ 3779 batches |  ms/batch 46789.05 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  3200/ 3779 batches |  ms/batch 47745.09 |  loss_dice  0.53  |  loss_gdl  0.53  |  dice_score  0.47  | 
| epoch   2 |  3300/ 3779 batches |  ms/batch 45802.81 |